In [58]:
import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import datetime
import re
import random

In [48]:
df = pd.read_csv('tweets_public.csv', encoding='utf-16', index_col='tweet_id', sep=',')
df.columns.tolist()

['airline_sentiment',
 'is_reply',
 'reply_count',
 'retweet_count',
 'text',
 'tweet_coord',
 'tweet_created',
 'tweet_location',
 'user_timezone']

In [49]:
weekdays = pd.to_datetime(df1.tweet_created).dt.dayofweek
hoursday = pd.to_datetime(df1.tweet_created).dt.hour
df1['weekday'] = pd.Series(weekdays, index=df1.index)
df1['hoursday'] = pd.Series(hoursday, index=df1.index)
weekday_names = []
wn = ['lunes', 'martes', 'miercoles', 'jueves', 'viernes', 'sabado', 'domingo']
for d in df1.weekday:
    weekday_names.append(wn[d])
df1['weekday_names'] = pd.Series(weekday_names, index=df1.index)
df1.tweet_created = pd.to_datetime(df1.tweet_created)

In [12]:
def create_submit_file(df_submission, ypred):
    date = datetime.datetime.now().strftime("%m_%d_%Y-%H_%M_%S")
    filename = 'submission_' + date + '.csv'

    df_submission['airline_sentiment'] = ypred
    df_submission[['airline_sentiment']].to_csv(filename)

    print('Submission file created: {}'.format(filename))
    print('Upload it to Kaggle InClass')

def stemlematizer(df):
    """ Function to reduce the words to their root
    """
    lemmer=WordNetLemmatizer()
    stemmer = SnowballStemmer('spanish')

    new_corpus1 = [' '.join([stemmer.stem(word) for word in tweet.split(' ')]) for tweet in df.text.values]
    new_corpus = [' '.join([lemmer.lemmatize(word) for word in tweet.split(' ')]) for tweet in new_corpus1] 
    
    df.text = new_corpus
    return df



def load_and_train(dataframe, trainmodelclassifier, test=None, *model_args, **model_kwargs ):

# If there is no test data, split the input
    if test is None:
        train, test = train_test_split(dataframe, test_size=0.15)
    else:
        train = dataframe

    dataframe.airline_sentiment = pd.Categorical(dataframe.airline_sentiment)
    
    x_train = train['text']
    y_train = train['airline_sentiment']
    
    x_test = test['text']
    #we create the list of stop words specific for spanish:
    spanish_stop_words = stopwords.words('spanish')
    #we build the classifier, that has everything inside:
    text_clf = Pipeline([('vect', CountVectorizer(min_df=1,
                                              stop_words=spanish_stop_words, ngram_range=(1,2), 
                                              analyzer='word', token_pattern=r'[^@]\b\w+\b')),
                     ('tfidf', TfidfTransformer(norm='l2')),
                     ('clf', trainmodelclassifier(*model_args, **model_kwargs))])

    

# SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None))])

    text_clf.fit(x_train, y_train)
    predicted = text_clf.predict(x_test)

    try:
        y_test = test['airline_sentiment'].values
    except:
        # It might be the submision file, where we don't have target values
        y_test = None
   
    try:
        acc = accuracy_score(test.airline_sentiment, predicted)
        print('accuracy_score = ', acc)
    except:
        print('No accuracy computed beacuse there is NO test target')
        acc = None
    return predicted, x_train, y_train, x_test, y_test, acc


In [13]:
def processTweet2(tweet):
    # process the tweets

    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #tweet = re.sub(r'\#\w+','',tweet)
    return tweet  

def handle_emojis(tweet):
    tweet = re.sub(r'CC:', ' EMOPOS ', tweet)
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMOPOS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMOPOS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMOPOS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMOPOS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMONEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMONEG ', tweet)
    return tweet

def preprocess_word(word):
    # Remove punctuation
    #word = word.strip('\'"?!,.():;')
    word = word.strip('\'"?!,.')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    # remove numbers
    word = re.sub(r'\d+','',word)
    #remove users 
    word = re.sub(r'AT_USER','',word)
    word = re.sub(r'URL','',word)
    word = re.sub(r'rt','',word)
    word = re.sub(r'via','',word)
    return word

def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

In [50]:
tweets = [handle_emojis(tweet) for tweet in df1.text.values]
tweets2 = [processTweet2(tweet) for tweet in tweets]
tweets3 = [' '.join(preprocess_word(word) for word in tweet.split(' ')) for tweet in tweets2]
df1.text = tweets3
df1.text.values
df1 = stemlematizer(df1)

In [34]:
hh = ['0','1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']
hh_names = ['zero', 'una', 'dos', 'tres', 'cuatro', 'cinco', 'seis', 'siete', 'ocho', 'nueve', 'diez', 'once', 'doce', 'trece', 'catorce', 'quince', 'dieciseis', 'diecisiete', 'dieciocho', 'diecinueve', 'veinte', 'veintiuno', 'veintios', 'veintitres']

In [51]:
#we create the list of stop words specific for spanish:
spanish_stop_words = stopwords.words('spanish')
count_vect = CountVectorizer(min_df=1, stop_words=newstopWords, ngram_range=(1,2), analyzer='word') #, token_pattern=r'[^@]\b\w+\b')
X_train_counts = count_vect.fit_transform(df1.text.values)
voc = count_vect.vocabulary_
X_train_counts = X_train_counts.toarray()

In [52]:
for i in hh_names:
    ind = df1.query('hoursday == @i').index
    df1.loc[ind,i] = 1

In [54]:
for h in hh_names:
    col = np.array(df1[h].fillna(value = 0))
    np.column_stack((X_train_counts,col))   

In [55]:
for i in np.unique(np.array(df1['weekday_names'])):
    ind = df1.query('weekday_names == @i').index
    df1.loc[ind,i] = 1
for d in wn:
    col = np.array(df1[d].fillna(value = 0))
    np.column_stack((X_train_counts,col))


In [59]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
features_train,features_test,label_train,label_test = \
train_test_split(X_train_tfidf, df1.airline_sentiment.values,test_size=0.15,random_state = 123)
sgc = SGDClassifier(penalty = 'elasticnet',max_iter = 10)
svm = LinearSVC()
sgc.fit(features_train,label_train)
svm.fit(features_train,label_train)
score = sgc.score(features_test,label_test)
scoresvm = svm.score(features_test,label_test)
print(score)
print(scoresvm)

0.613039796782
0.602878916173


Here I do the processing of the sampleSubmission.csv file, with the created model above.

In [ ]:
df0_s = pd.read_csv('tweets_submission.csv', index_col='tweet_id', sep=',')
# Keep relevant columns only
df_s = df0_s.drop(['retweet_count','is_reply','reply_count','tweet_coord','tweet_location','user_timezone'], axis=1)
#df.head()
df_s['text'] = df_s['text'].apply(basicCleaning)
df_s['airline'] = df_s['text'].apply(extract_airline)
# From 'airline' column create a one-hot encoding matrix for airline name
tweet_airline = df_s['airline'].str.split(r'|', expand=True).stack().reset_index(level='tweet_id')
tweet_airline.columns = ['tweet_id','airline']
tweet_airline = tweet_airline.set_index('tweet_id')
# One-hot encoding for airline name
onehot = pd.get_dummies(tweet_airline['airline'])
df_airlines = onehot.groupby('tweet_id').sum()

# Concatenate the two dataframes (original+one-hot airlines) and delete the airline column
merged = pd.concat([df_s, df_airlines], axis=1, join_axes=[df_s.index])
df1_s = merged.drop('airline', axis=1)
df1_s['text'] = df1_s['text'].apply(moreCleaning)

weekdays = pd.to_datetime(df1_s.tweet_created).dt.dayofweek
hoursday = pd.to_datetime(df1_s.tweet_created).dt.hour
df1_s['weekday'] = pd.Series(weekdays, index=df1_s.index)
df1_s['hoursday'] = pd.Series(hoursday, index=df1_s.index)
weekday_names = []
wn = ['lunes', 'martes', 'miercoles', 'jueves', 'viernes', 'sabado', 'domingo']
for d in df1_s.weekday:
    weekday_names.append(wn[d])
df1_s['weekday_names'] = pd.Series(weekday_names, index=df1_s.index)
df1_s.tweet_created = pd.to_datetime(df1_s.tweet_created)



tweets = [handle_emojis(tweet) for tweet in df1_s.text.values]
tweets2 = [processTweet2(tweet) for tweet in tweets]
tweets3 = [' '.join(preprocess_word(word) for word in tweet.split(' ')) for tweet in tweets2]
df1_s.text = tweets3
df1_s.text.values
df1_s = stemlematizer(df1_s)

count_vect = CountVectorizer(vocabulary=voc, min_df=1, stop_words=newstopWords, ngram_range=(1,2), analyzer='word')
X_submission_counts = count_vect.fit_transform(df1_s.text.values)
X_submission_counts = X_submission_counts.toarray()

for i in hh_names:
    ind = df1_s.query('hoursday == @i').index
    df1_s.loc[ind,i] = 1

for h in hh_names:
    col = np.array(df1_s[h].fillna(value = 0))
    np.column_stack((X_submission_counts,col))    
    
for i in np.unique(np.array(df1_s['weekday_names'])):
    ind = df1_s.query('weekday_names == @i').index
    df1_s.loc[ind,i] = 1
for d in wn:
    col = np.array(df1_s[d].fillna(value = 0))
    np.column_stack((X_submission_counts,col))

X_submission_tfidf = tfidf_transformer.transform(X_submission_counts)
#features_train,features_test,label_train,label_test = \
#train_test_split(X_submission_tfidf, df1_s.airline_sentiment.values,test_size=1.00,random_state = 123)
#sgc = SGDClassifier(penalty = 'elasticnet',max_iter = 10)
#svm = LinearSVC()
#sgc.fit(features_train,label_train)
#svm.fit(features_train,label_train)
prediction_sgc = sgc.predict(X_submission_tfidf)


In [ ]:
create_submit_file(df0_s, prediction_sgc)